### Example of a "self-healing" AI pipeline. 
It moves beyond just asking an AI a question and starts treating it like a reliable software module.

In [1]:
from typing import List
from pydantic import BaseModel, Field, field_validator
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnableLambda


# 1. THE CONTRACT (Output Schema)
class PatientSummary(BaseModel):
    patient_name: str = Field(description="Full name of the patient")
    age: int = Field(description="Age in years")
    diagnoses: List[str] = Field(description="List of medical conditions")

    @field_validator("age")
    @classmethod
    def check_age(cls, v: int) -> int:
        # Custom business logic that Pydantic's basic types can't catch
        if v < 0 or v > 120:
            raise ValueError("Age must be between 0 and 120")
        return v


# 2. SETUP
# The Parser is the bridge: it generates the instructions for the LLM
# and knows how to turn a raw string back into a Python object.
parser = PydanticOutputParser(pydantic_object=PatientSummary)

# Temperature 0 is critical for data extraction—it minimizes "hallucinations"
# and makes the output more deterministic (consistent).
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# .partial() injects the 'format_instructions' early so you don't have
# to pass them manually every time you call the chain.
# The parser.get_format_instructions() literally tells the LLM:
# "Output a JSON object with these specific keys..."
prompt = ChatPromptTemplate.from_template(
    "Extract info from this note.\n{format_instructions}\nNote: {note}"
).partial(format_instructions=parser.get_format_instructions())


# 3. THE REPAIR LOGIC (The "Self-Healing" Function)
def validate_and_repair(output):
    """
    This function acts as a 'middleware' layer.
    It receives the LLM's text and attempts to force it into our Pydantic model.
    """
    try:
        # parser.parse() takes the string from 'output.content' and
        # runs Pydantic's validation checks on it.
        return parser.parse(output.content)
    except Exception as e:
        # If the LLM gives us bad JSON or a negative age, we catch the error here.
        print(f"\n [CONTRACT VIOLATION]: {e}")

        # REPAIR STRATEGY: We send the specific error back to the LLM.
        # This 'Feedback Loop' allows the AI to see why it failed and try again.
        repair_msg = (
            f"Your last response broke the contract: {e}. "
            "Return ONLY the raw JSON. No conversational text, no markdown blocks."
        )

        # We create a mini-chain on the fly to process the repair.
        # The '|' (pipe) operator merges the LLM and Parser into one unit.
        repair_chain = llm | parser
        return repair_chain.invoke(repair_msg)


# 4. THE CHAIN (The Assembly Line)
# This uses LCEL (LangChain Expression Language).
# Data flows: Prompt -> LLM -> validate_and_repair function.
chain = prompt | llm | RunnableLambda(validate_and_repair)

# 5. EXECUTION
# This note is designed to fail (age -10), triggering our @field_validator.
messy_note = "Subject is Benjamin Button. He claims to be -10 years old due to a temporal anomaly."

# .invoke() starts the flow.
result = chain.invoke({"note": messy_note})

print("\n Final Validated Object (Executable Data):")
# Because 'result' is now a Pydantic object (not just a string),
# we can use dot notation to access attributes safely.
print(f"Name: {result.patient_name}")
print(f"Age: {result.age}")
print(f"Diagnoses: {result.diagnoses}")


 [CONTRACT VIOLATION]: Failed to parse PatientSummary from completion {"patient_name": "Benjamin Button", "age": -10, "diagnoses": ["temporal anomaly"]}. Got: 1 validation error for PatientSummary
age
  Value error, Age must be between 0 and 120 [type=value_error, input_value=-10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 

 Final Validated Object (Executable Data):
Name: Benjamin Button
Age: 10
Diagnoses: ['temporal anomaly']
